[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/python35/IINTS-SDK/blob/main/examples/notebooks/04_Baseline_and_Metrics.ipynb)


In [ ]:
import os
import sys
from pathlib import Path
import subprocess

def _ensure_repo():
    workspace = os.environ.get('GITHUB_WORKSPACE')
    if workspace:
        repo_root = Path(workspace).resolve()
        src_path = repo_root / 'src'
        if src_path.exists():
            sys.path.insert(0, str(src_path))
            return repo_root
    repo_root = Path().resolve()
    src_path = repo_root / 'src'
    if src_path.exists():
        sys.path.insert(0, str(src_path))
        return repo_root
    repo_dir = Path('IINTS-SDK')
    if not repo_dir.exists():
        subprocess.check_call(['git', 'clone', 'https://github.com/python35/IINTS-SDK.git'])
    repo_root = repo_dir.resolve()
    os.chdir(repo_root)
    src_path = repo_root / 'src'
    sys.path.insert(0, str(src_path))
    return repo_root

_ensure_repo()


# Baseline Comparison and Metrics

Compare your algorithm to the built-in baseline and compute clinical metrics.


In [ ]:
import json
from pathlib import Path

import iints
from iints.core.algorithms.mock_algorithms import ConstantDoseAlgorithm
from iints.metrics import calculate_full_metrics

presets = json.loads(Path("src/iints/presets/presets.json").read_text())
preset = next(p for p in presets if p["name"] == "baseline_t1d")

outputs = iints.run_simulation(
    algorithm=ConstantDoseAlgorithm,
    scenario=preset["scenario"],
    patient_config=preset["patient_config"],
    duration_minutes=240,
    time_step=preset["time_step_minutes"],
    seed=21,
    output_dir="results/baseline_metrics",
    compare_baselines=True,
)

comparison = outputs["baseline_comparison"]
comparison


In [ ]:
glucose = outputs["results"]["glucose_actual_mgdl"]
calculate_full_metrics(glucose)
